# Exploratory Data Analysis

## Objective :
- Delete missing values
- Perform relevant statistical tests
- Encode categorical variables
- Standardize numerical variables

In [ ]:
# Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import scipy.stats as stats
from scipy.stats import chi2_contingency

from imblearn.over_sampling import SMOTE

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
import xgboost as xgb

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score, roc_curve, recall_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, randint

from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay

import pickle

In [2]:
# Load the DataFrame
df = pd.read_pickle('Exploration_data.pkl')

df

,Customer Number,Offer Accepted,Reward,Mailer Type,Income Level,# Bank Accounts Open,Overdraft Protection,Credit Rating,# Credit Cards Held,# Homes Owned,Household Size,Own Your Home,Average Balance,Q1 Balance,Q2 Balance,Q3 Balance,Q4 Balance
0,1,No,Air Miles,Letter,High,1,No,High,2,1,4,No,1160.75,1669.0,877.0,1095.0,1002.0
1,2,No,Air Miles,Letter,Medium,1,No,Medium,2,2,5,Yes,147.25,39.0,106.0,78.0,366.0
2,3,No,Air Miles,Postcard,High,2,No,Medium,2,1,2,Yes,276.50,367.0,352.0,145.0,242.0
3,4,No,Air Miles,Letter,Medium,2,No,High,1,1,4,No,1219.00,1578.0,1760.0,1119.0,419.0
4,5,No,Air Miles,Letter,Medium,1,No,Medium,2,1,6,Yes,1211.00,2140.0,1357.0,982.0,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,17996,No,Cash Back,Letter,High,1,No,Low,1,1,5,Yes,167.50,136.0,65.0,71.0,398.0
17996,17997,No,Cash Back,Letter,High,1,No,Low,3,1,3,Yes,850.50,984.0,940.0,943.0,535.0
17997,17998,No,Cash Back,Letter,High,1,No,Low,2,1,4,No,1087.25,918.0,767.0,1170.0,1494.0
17998,17999,No,Cash Back,Letter,Medium,1,No,Medium,4,2,2,Yes,1022.25,626.0,983.0,865.0,1615.0


## Let's confirm the hypotheses that the graphs gave us with statistical tests

### Handling missing values

In [4]:
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_table = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percent})
print(missing_table)

                      Missing Values  Percentage
Customer Number                    0    0.000000
Offer Accepted                     0    0.000000
Reward                             0    0.000000
Mailer Type                        0    0.000000
Income Level                       0    0.000000
# Bank Accounts Open               0    0.000000
Overdraft Protection               0    0.000000
Credit Rating                      0    0.000000
# Credit Cards Held                0    0.000000
# Homes Owned                      0    0.000000
Household Size                     0    0.000000
Own Your Home                      0    0.000000
Average Balance                   24    0.133333
Q1 Balance                        24    0.133333
Q2 Balance                        24    0.133333
Q3 Balance                        24    0.133333
Q4 Balance                        24    0.133333


In [5]:
# Select all rows containing NaN values
nan_rows = df[df.isnull().any(axis=1)]

# Show lines containing NaNs
pd.DataFrame(nan_rows)

,Customer Number,Offer Accepted,Reward,Mailer Type,Income Level,# Bank Accounts Open,Overdraft Protection,Credit Rating,# Credit Cards Held,# Homes Owned,Household Size,Own Your Home,Average Balance,Q1 Balance,Q2 Balance,Q3 Balance,Q4 Balance
323,324,No,Points,Postcard,Medium,1,No,High,1,1,4,No,NaN,NaN,NaN,NaN,NaN
3070,3071,Yes,Air Miles,Postcard,Low,3,No,Medium,2,1,4,Yes,NaN,NaN,NaN,NaN,NaN
3108,3109,No,Air Miles,Letter,Low,1,No,High,3,1,4,Yes,NaN,NaN,NaN,NaN,NaN
3789,3790,No,Air Miles,Letter,Medium,2,No,High,3,1,3,No,NaN,NaN,NaN,NaN,NaN
5083,5084,No,Points,Letter,Medium,1,No,High,1,1,3,Yes,NaN,NaN,NaN,NaN,NaN
5677,5678,No,Air Miles,Postcard,Low,1,No,Low,2,1,4,Yes,NaN,NaN,NaN,NaN,NaN
5689,5690,No,Air Miles,Postcard,Medium,1,No,Medium,3,1,5,No,NaN,NaN,NaN,NaN,NaN
6932,6933,No,Air Miles,Postcard,Medium,1,No,Medium,2,2,4,Yes,NaN,NaN,NaN,NaN,NaN
7787,7788,No,Air Miles,Letter,Low,2,No,Low,1,1,4,Yes,NaN,NaN,NaN,NaN,NaN
7864,7865,No,Cash Back,Postcard,High,2,Yes,Low,2,1,3,No,NaN,NaN,NaN,NaN,NaN


In [6]:
pd.DataFrame(nan_rows).shape

(24, 17)

Interpretation: we can see here that the missing values are strongly correlated with each other, because they are linked to each other, we will therefore delete them because they only represent 0.133% of the data for each variable.

In [7]:
#drop 'NaN'
df = df.dropna(axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17976 entries, 0 to 17999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer Number       17976 non-null  int64  
 1   Offer Accepted        17976 non-null  object 
 2   Reward                17976 non-null  object 
 3   Mailer Type           17976 non-null  object 
 4   Income Level          17976 non-null  object 
 5   # Bank Accounts Open  17976 non-null  int64  
 6   Overdraft Protection  17976 non-null  object 
 7   Credit Rating         17976 non-null  object 
 8   # Credit Cards Held   17976 non-null  int64  
 9   # Homes Owned         17976 non-null  int64  
 10  Household Size        17976 non-null  int64  
 11  Own Your Home         17976 non-null  object 
 12  Average Balance       17976 non-null  float64
 13  Q1 Balance            17976 non-null  float64
 14  Q2 Balance            17976 non-null  float64
 15  Q3 Balance            17

In [8]:
from scipy.stats import chi2_contingency

#initialer les listes pour stocker les resultats
var_names = []
chi2_stats = []
p_values = []
cramer_vs = []

#Parcourir toutes les variables categorielles
for var in df.select_dtypes('object'):
    #calculer le tableau de contingence
    contingenc_table = pd.crosstab(df['Offer Accepted'], df[var])
    #calculer la statistique de test du chi-deux et le p-value
    chi2, p, dof, expected = chi2_contingency(contingenc_table)
    #calcul le coefficient V de cramer
    n = contingenc_table.sum().sum()
    phi2 = chi2/n
    r,k = contingenc_table.shape
    phi2corr = max(0, phi2-(k-1)*(r-1))/(n-1)
    rc = r-((r-1)**2)/(n-1)
    kc = k-((k-1)**2)/(n-1)
    cramer_v = np.sqrt(phi2-((k-1)*(r-1))/(n-1))
    #ajouter les resultats aux listes correspondantes
    var_names.append(var)
    chi2_stats.append(chi2)
    p_values.append(p)
    cramer_vs.append(cramer_v)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9832\2197605215.py:22: RuntimeWarning: invalid value encountered in sqrt
  cramer_v = np.sqrt(phi2-((k-1)*(r-1))/(n-1))
C:\Users\Admin\AppData\Local\Temp\ipykernel_9832\2197605215.py:22: RuntimeWarning: invalid value encountered in sqrt
  cramer_v = np.sqrt(phi2-((k-1)*(r-1))/(n-1))


In [9]:
# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Variable': var_names,
    'Chi2_statistic': chi2_stats,
    'P_value': p_values,
    "Cramer's_V": cramer_vs
})


# Sort the results DataFrame by Cramer's V coefficient in ascending order
sorted_results_df = results_df.sort_values(by="Cramer's_V", ascending=True)

# Display the sorted results
print(sorted_results_df)


               Variable  Chi2_statistic       P_value  Cramer's_V
1                Reward      101.147670  1.086582e-22    0.074267
3          Income Level      113.617605  2.129367e-25    0.078799
2           Mailer Type      169.039505  1.199365e-38    0.096685
5         Credit Rating      445.062142  2.269780e-97    0.156995
0        Offer Accepted    17957.338359  0.000000e+00    0.999453
4  Overdraft Protection        0.099898  7.519518e-01         NaN
6         Own Your Home        0.000000  1.000000e+00         NaN


In [10]:
import scipy.stats as stats

# Initialize lists to store the results
var_names = []
kw_stats = []
p_values = []

# Iterate over all numerical variables
for var in df.select_dtypes(['float', 'int']):
    # Calculate the groups of values
    groups = [df[df['Offer Accepted'] == 'No'][var], df[df['Offer Accepted'] == 'Yes'][var]]
    # Apply the Kruskal-Wallis test
    kw_stat, p = stats.kruskal(*groups)
    # Add the results to the corresponding lists
    var_names.append(var)
    kw_stats.append(kw_stat)
    p_values.append(p)

# Create a DataFrame with the results
results_df = pd.DataFrame({
    'Variable': var_names,
    'Kruskal-Wallis': kw_stats,
    'P-value': p_values
})

# Sort the DataFrame in ascending order of p-value
results_df.sort_values(by='P-value', inplace=True)

# Display the results table
print(results_df)

               Variable  Kruskal-Wallis   P-value
6            Q1 Balance        5.997898  0.014323
4        Household Size        2.741845  0.097752
8            Q3 Balance        1.075692  0.299663
9            Q4 Balance        0.145636  0.702741
2   # Credit Cards Held        0.118703  0.730445
7            Q2 Balance        0.067323  0.795276
0       Customer Number        0.016807  0.896849
5       Average Balance        0.015696  0.900299
1  # Bank Accounts Open        0.001363  0.970551
3         # Homes Owned        0.000988  0.974931


#### Interpretation: This information will be useful if I need to make a selection of variables for my model.

## Encoding categorical variables

In [11]:
label_encoder = LabelEncoder()

In [12]:
le_reward = LabelEncoder()
df['Reward'] = le_reward.fit_transform(df['Reward'])
df.Reward.unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_9832\2933253246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Reward'] = le_reward.fit_transform(df['Reward'])


array([0, 1, 2])

In [13]:
le_mailerType = LabelEncoder()
df['Mailer Type'] = le_mailerType.fit_transform(df['Mailer Type'])
df['Mailer Type'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_9832\2463393731.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Mailer Type'] = le_mailerType.fit_transform(df['Mailer Type'])


array([0, 1])

In [14]:
le_incomeLevel = LabelEncoder()
df['Income Level'] = le_incomeLevel.fit_transform(df['Income Level'])
df['Income Level'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_9832\1317214983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Income Level'] = le_incomeLevel.fit_transform(df['Income Level'])


array([0, 2, 1])

In [15]:
le_overdraftProtection = LabelEncoder()
df['Overdraft Protection'] = le_overdraftProtection.fit_transform(df['Overdraft Protection'])
df['Overdraft Protection'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_9832\3875484494.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Overdraft Protection'] = le_overdraftProtection.fit_transform(df['Overdraft Protection'])


array([0, 1])

In [16]:
le_creditRating = LabelEncoder()
df['Credit Rating'] = le_creditRating.fit_transform(df['Credit Rating'])
df['Credit Rating'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_9832\3712157592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Credit Rating'] = le_creditRating.fit_transform(df['Credit Rating'])


array([0, 2, 1])

In [17]:
le_homeOwner = LabelEncoder()
df['Own Your Home'] = le_homeOwner.fit_transform(df['Own Your Home'])
df['Own Your Home'].unique()

C:\Users\Admin\AppData\Local\Temp\ipykernel_9832\946911161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Own Your Home'] = le_homeOwner.fit_transform(df['Own Your Home'])


array([0, 1])

## Standardization

### It will be applied in the next step after the data is divided into train and test

In [18]:
df

,Customer Number,Offer Accepted,Reward,Mailer Type,Income Level,# Bank Accounts Open,Overdraft Protection,Credit Rating,# Credit Cards Held,# Homes Owned,Household Size,Own Your Home,Average Balance,Q1 Balance,Q2 Balance,Q3 Balance,Q4 Balance
0,1,No,0,0,0,1,0,0,2,1,4,0,1160.75,1669.0,877.0,1095.0,1002.0
1,2,No,0,0,2,1,0,2,2,2,5,1,147.25,39.0,106.0,78.0,366.0
2,3,No,0,1,0,2,0,2,2,1,2,1,276.50,367.0,352.0,145.0,242.0
3,4,No,0,0,2,2,0,0,1,1,4,0,1219.00,1578.0,1760.0,1119.0,419.0
4,5,No,0,0,2,1,0,2,2,1,6,1,1211.00,2140.0,1357.0,982.0,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,17996,No,1,0,0,1,0,1,1,1,5,1,167.50,136.0,65.0,71.0,398.0
17996,17997,No,1,0,0,1,0,1,3,1,3,1,850.50,984.0,940.0,943.0,535.0
17997,17998,No,1,0,0,1,0,1,2,1,4,0,1087.25,918.0,767.0,1170.0,1494.0
17998,17999,No,1,0,2,1,0,2,4,2,2,1,1022.25,626.0,983.0,865.0,1615.0


In [19]:
# Save the preprocessed DataFrame
df.to_pickle('processed_data.pkl')